In [6]:
from collections import defaultdict

class CTFeatureExtraction:
    def __init__(self):
        self.amino_acid_groups = {
            1: 'AVG',
            2: 'TSYM',
            3: 'FLIP',
            4: 'HQNW',
            5: 'DE',
            6: 'RK',
            7: 'C'
        }

    def calculate_ct_features(self, sequence):
        """
        Calculate the CT features for a given protein sequence.

        :param sequence: The protein sequence as a string.
        :return: A list of CT feature values.
        """
        # Create a mapping of amino acids to their corresponding groups
        aa_to_group = {}
        for group, amino_acids in self.amino_acid_groups.items():
            for aa in amino_acids:
                aa_to_group[aa] = group

        # Initialize a dictionary to store the counts of each triad
        triad_counts = defaultdict(int)

        # Iterate over the sequence and count the occurrences of each triad
        for i in range(len(sequence) - 2):
            triad = ''.join([str(aa_to_group.get(aa, 0)) for aa in sequence[i:i+3]])
            triad_counts[triad] += 1

        # Calculate the total number of triads
        total_triads = sum(triad_counts.values())

        # Initialize a list to store the CT feature values
        ct_features = []

        # Calculate the CT feature values for each possible triad
        for i in range(1, len(self.amino_acid_groups) + 1):
            for j in range(1, len(self.amino_acid_groups) + 1):
                for k in range(1, len(self.amino_acid_groups) + 1):
                    triad = f'{i}{j}{k}'
                    if total_triads > 0:
                        ct_features.append(triad_counts[triad] / total_triads)
                    else:
                        ct_features.append(0)

        return ct_features

# Example usage
# sequence = "ACDEFGHIKLMNPQRSTVWY"
# extractor = CTFeatureExtraction()
# ct_features = extractor.calculate_ct_features(sequence)
# print(ct_features)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Load the dataset
data = pd.read_excel('../data/Final_2Sm_modified_with_sequences.xlsx')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to the folding_type column and transform it to numeric labels
data['folding_type'] = label_encoder.fit_transform(data['folding_type'])

# Now, when you extract labels for model training:
labels = data['folding_type'].values

# Initialize the FeatureExtraction class
feature_extraction = CTFeatureExtraction()

# Feature extraction using AAC with length
features = np.array([feature_extraction.calculate_ct_features(seq) for seq in data['sequence']])

# Support Vector Machine (SVM) Classifier

In [4]:
# SVM with Leave-One-Out Cross-Validation (LOOCV)
loo = LeaveOneOut()
y_true, y_pred = [], []
for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred.append(clf.predict(X_test)[0])
    y_true.append(y_test[0])

In [5]:
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'KAAC')
cm.evaluate()

Confusion Matrix: $KAAC
[[89  0]
 [52  0]]

Accuracy (ACC): 0.63
Matthews Correlation Coefficient (MCC): 0.00

Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        89
           1       0.00      0.00      0.00        52

    accuracy                           0.63       141
   macro avg       0.32      0.50      0.39       141
weighted avg       0.40      0.63      0.49       141


# Random Forest Classifier implementation with Hyperparameter Tuning

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Grid search with cross validation setup
grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, scoring='accuracy')

# Fit the grid search to find the best parameters
grid_search.fit(features, labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters: ", best_params)
print("Best score: ", best_score)

# Using the best parameters with LOOCV
best_rf = RandomForestClassifier(**best_params, random_state=42)
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    best_rf.fit(X_train, y_train)
    y_pred.append(best_rf.predict(X_test)[0])
    y_true.append(y_test[0])

Best parameters:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best score:  0.7234042553191489


In [9]:
# Evaluate the model
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'CT')
cm.evaluate()

Confusion Matrix: $CT
[[83  6]
 [31 21]]

Accuracy (ACC): 0.74
Matthews Correlation Coefficient (MCC): 0.41

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.93      0.82        89
           1       0.78      0.40      0.53        52

    accuracy                           0.74       141
   macro avg       0.75      0.67      0.67       141
weighted avg       0.75      0.74      0.71       141
